In [1]:
# Carga de librerias
import numpy as np 
import pandas as pd 
import os
import seaborn as sns
import matplotlib.pyplot as plt
import cv2
import glob
import os
from PIL import Image
from itertools import chain
from collections import Counter
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.layers.experimental.preprocessing import Resizing
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.preprocessing.image import DirectoryIterator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import accuracy_score
from keras import backend as K
from itertools import chain
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.densenet import DenseNet121
# Definición de las variables globales
IMG_SIZE=32
NUM_CHANNELS=3
NUM_CLASSES=43
from random import sample, seed
seed(3)

In [3]:
# Declaración de los directorios
current_dir=os.getcwd()
dataset_dir=current_dir+"/Dataset"
dir_train=dataset_dir+"/Train"
train_data=pd.read_csv(dataset_dir+"/Train.csv")
test_data=pd.read_csv(dataset_dir+"/Test.csv")
meta_data=pd.read_csv(dataset_dir+"/Meta.csv")

x_images=test_data["Path"]
x_train_images=train_data["Path"]
x_train_labels=train_data[['ClassId']].to_numpy()
y_test=test_data[['ClassId']].values

In [ ]:
path_180="Train_180"
path_80="Train_80"
path_combined="Train_combined"
dir_train_180=dataset_dir+"/"+path_180
dir_train_80=dataset_dir+"/"+path_80
dir_train_combined=dataset_dir+"/"+path_combined
copy_tree(dir_train,dir_train_180)
copy_tree(dir_train,dir_train_80)
copy_tree(dir_train,dir_train_combined)

In [5]:
def prepare_train_images(path,brig, x_train_images comb=False):
    
    if comb==False:
        for name in x_train_images:
            im=name.replace("Train", path)
            image=cv2.imread(dataset_dir+"/"+im)
            image=change_brightness(image, brig)
            cv2.imwrite(im,image)
    else:
        for i,name in enumerate(x_train_images):
            im=name.replace("Train", path)
            image=cv2.imread(im)
            if i<=len(x_train_images_2)/2:
                image=change_brightness(image, -80)
            else:
                image=change_brightness(image, 180)
            cv2.imwrite(im,image)
    path="./"+path
    return path


In [6]:
# Inception
def train_data(size,shear,rotation,brig,dir_train_brig):
    # Definimos el data generator
    train_datagen = ImageDataGenerator(
        rescale=1 / 255.0,
        validation_split=0.2,
        shear_range=shear,
        rotation_range=rotation,
        brightness_range=brig
    )

    # Generamos los datos de entrenamiento
    train_generator = train_datagen.flow_from_directory(
        directory=dir_train_brig,
        target_size=size,
        class_mode="categorical",
        subset='training',
        shuffle=True,
        seed=3)


    # Generamos los datos de vaidacion
    valid_generator = train_datagen.flow_from_directory(
        directory=dir_train_brig,
        target_size=size,
        class_mode="categorical",
        subset='validation',
        shuffle=True,
        seed=3)
    return train_generator,valid_generator

In [7]:
def weights_train(train_data):
    # Contamos el numero de valores por clase
    counter = Counter(train_data.classes)
    # Almacenamos los valores máximos
    max_val = float(max(counter.values()))
    # Calculamos los pesos
    class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}                     
    return class_weights

In [8]:
settings = {
    'inceptionV3': {
        'model': InceptionV3,
        'img_size': (150, 150),
        'shear': 0.2,
        'brig': None,
        'rotation': None
    },
    'resnet50': {
        'model': ResNet50,
        'img_size': (224, 224),
        'shear': 0.2,
        'brig': [0.8,0.8],
        'rotation': None   
    },
    'DenseNet121': {
        'model': DenseNet121,
        'img_size': (224, 224),
        'shear': 0.2,
        'brig': None,
        'rotation': None    }
}

In [9]:
def test_generator(test_data,img_size,brig,comb=False):
    # Recorremos todas las imágenes
    for i,image_path in enumerate(test_data):
        if comb==False:
            
            # Para cada imagen realizamos un resizing y lo convertimos en un array 
            image = load_img(dir_dataset+"/"+image_path, target_size=img_size)
            image_array = img_to_array(image)
            image_array=change_brightness(image_array,brig)
            image_array=image_array/255
            image_array = np.array([image_array])
        else:
            if i<=len(test_data)/2:
                image = load_img(dir_dataset+"/"+image_path, target_size=img_size)
                image_array = img_to_array(image)
                image_array=change_brightness(image_array,180)
                image_array=image_array/255
                image_array = np.array([image_array])
            else:
                image = load_img(dir_dataset+"/"+image_path, target_size=img_size)
                image_array = img_to_array(image)
                image_array=change_brightness(image_array,-80)
                image_array=image_array/255
                image_array = np.array([image_array])
                
        # Pasamos la imagen a la función de preprocesado utilizada en el modelo
        yield image_array

In [10]:
def change_brightness(img, value):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)
    v = cv2.add(v,value)
    v[v > 255] = 255
    v[v < 0] = 0
    final_hsv = cv2.merge((h, s, v))
    img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return img

In [11]:
def predictions (model,size,x_images, valid,bri,combinations=False):
    results=[]
    models=[]
    
    if combinations==False:
        X_test=test_generator(x_images,size,bri,False)
    else:
        X_test=test_generator(x_images,size,bri,True)

    y_pred = model.predict(X_test)
    y_pred=np.argmax(y_pred,axis=1)
        
    label_mapper = {v: k for k, v in valid.class_indices.items()}
    y_pred = list(map(lambda x: label_mapper[x], y_pred))
    val_acc=accuracy_score(y_pred,y_test)
        
    params=model.count_params()
        
    return val_acc,params

In [12]:
def retrain_imagenet_models (settings,dir_train_brig,bri,combinations=False):
    results=[]
    models=[]
    history_results=[]
    dir_train=prepare_train_images(dir_train_brig,bri, combinations)
    for name, model_settings in settings.items():
        model=model_settings['model']
        size=model_settings["img_size"]
        brig=model_settings["brig"]
        shear=model_settings["shear"]
        rotation=model_settings["rotation"]
        
        model_name=current_dir+'/Results/Pretrained_train/'+name+'.h5'
        h5_model=tf.keras.models.load_model(model_name)
        train,valid=train_data(size,shear,rotation,brig,dir_train)
        weights=weights_train(train)
        callback=EarlyStopping(monitor='val_loss', verbose=1, patience=6,restore_best_weights=True)
        # Entrenamos el modelo
        history = h5_model.fit(train ,validation_data=(valid), 
                                                  epochs=15, verbose=1, callbacks=[callback], class_weight=weights)
        history_results.append((history,name))
        
        val_acc,params=predictions(h5_model,size,x_images,valid,bri,combinations)
        
        tmp={"Model": name,
         "Accuracy": val_acc,
         "Number of parameters": params
             }
        results.append(tmp)
        model_name=current_dir+'/Results/Pretrained_defense/'+name+'.h5'
        h5_model.save(model_name)
        
        
    return history_results,pd.DataFrame(results)
    

In [13]:
def plot_history(history_model,brig):
    list_models=[]
    fig, (ax1) = plt.subplots(1,1, figsize=(12, 10))
    for i,(h,model_name) in enumerate(history_model):
        # Visualizamos la evolución de la accuracy
        ax1.plot(h.history['val_accuracy'])
        ax1.set_title("Brightness factor"+ str(brig))
        ax1.set_ylabel('Accuracy')
        ax1.set_xlabel('Epoch')
        list_models.append(str(model_name))
    fig.legend(list_models, loc="right")
# Visualizamos la evolución del error cometido por la red
    plt.show()

In [ ]:
history_80,result_80=retrain_imagenet_models(settings,path_80,-80,combinations=False)
plot_history(history_combined,"Factor -80")
result_80.to_csv(current_dir+'/Results/Pretrained_attack/Pre_Trained_retrained_80.csv')

history_180,result_180=retrain_imagenet_models(settings,path_180,180,combinations=False)
plot_history(history_combined,"Factori 180")
result_180.to_csv(current_dir+'/Results/Pretrained_attack/Pre_Trained_retrained_180.csv')

history_combined,result_combined=retrain_imagenet_models(settings,path_combined,180,combinations=True)
plot_history(history_combined,"Combined")
result_combined.to_csv(current_dir+'/Results/Pretrained_attack/Pre_Trained_retrained_combined.csv')